In [ ]:
import tensorflow as tf
import cv2
import numpy as np
import time
import os
from flask import Flask, render_template, request, send_from_directory
import json

app = Flask(__name__, static_url_path="")

IMAGE_SIZE = (224, 224, 3)
model = None

def predict_model(model, video):
    cap = cv2.VideoCapture(video)

    data_matrix = []

    # read in the frames
    while True:
        ret, frame = cap.read()
        if not ret:
            break

        image = cv2.resize(frame, (IMAGE_SIZE[0], IMAGE_SIZE[1])) # resize
        data_matrix.append(image)

    cur_time = time.time()
    prediction = model.predict(np.array([data_matrix]))
    pred_time = time.time() - cur_time
    print(prediction)
    text_pred = "Hand Flapping" if prediction >= 0.5 else "No Hand Flapping"

    cap.release()
    return round(pred_time, 2), (text_pred, round(max([(1 - prediction)[0][0], prediction[0][0]]), 2))

@app.before_first_request
def before_first_request():
    global model
    model = tf.keras.models.load_model("MBNet")

@app.route("/")
def main():
    return app.send_static_file('demo.html')

@app.route("/predict", methods=['POST', 'GET'])
def predict():
    data = request.get_json()
    file_name = data['file']
    if not file_name.endswith(".mov") and not file_name.endswith(".mp4"):
        return "Invalid File!"

    pred_time, (prediction_class, confidence) = predict_model(model, f"static/videos/{file_name}")
    print(pred_time, prediction_class, confidence)
    return {"time": str(pred_time), "prediction": prediction_class, "confidence": str(confidence)}

if __name__ == "__main__":
    #app.run(debug=True, port=8000, host="localhost",use_reloader=False)
    #app.run(host='localhost',port=4455,debug=True) 
    app.run(debug=True, port=8049, use_reloader=False)

    


 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


 * Running on http://127.0.0.1:8049
Press CTRL+C to quit
127.0.0.1 - - [04/Oct/2023 15:49:36] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [04/Oct/2023 15:49:36] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [04/Oct/2023 15:49:36] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [04/Oct/2023 15:49:36] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [04/Oct/2023 15:55:34] "GET /videos/v_ArmFlapping_01.xml HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [04/Oct/2023 15:55:34] "GET /videos/v_ArmFlapping_01.xml HTTP/1.1" 404 -
127.0.0.1 - - [04/Oct/2023 15:55:40] "GET /videos/v_ArmFlapping_02.xml HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [04/Oct/2023 15:55:40] "GET /videos/v_ArmFlapping_02.xml HTTP/1.1" 404 -
127.0.0.1 - - [04/Oct/2023 15:58:09] "GET /videos/pexels-mikhail-nilov-8655759%20(1080p).mp4 HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [04/Oct/2023 15:58:09] "GET /videos/pexels-mikhail-nilov-8655759%20(1080p).mp4 HTTP/1.1" 404 -
OpenCV: Couldn't read video stream from file 